In [1]:
# https://stackoverflow.com/questions/37159070/multiple-linear-regression-model-by-using-tensorflow
# https://donaldpinckney.com/books/pytorch/book/ch2-linreg/2018-03-21-multi-variable.html
# https://www.youtube.com/watch?v=Q4GNLhRtZNc
# https://atmamani.github.io/projects/ml/coursera-gd-multivariate-linear-regression/
# https://online.stat.psu.edu/stat462/sites/onlinecourses.science.psu.edu.stat462/files/05mlr/eq_matrix_notation/index.gif

# Multivariable Logistic Regression for matricies.
# target = flux1 + flux2 +... flux500 + b
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np
import pandas as pd
from numpy import exp
from sklearn import preprocessing
import pickle

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def labels(filename):
    data = np.matrix(pd.read_csv(filename).values)

    data_y = data[:, 0].transpose()
    data_y-=1 #binaryization of the categorical data
    return data_y

In [3]:
#Logistic Layer using a sigmoid function
def logistic_layer(y):
    y = np.array(y)
    y = 1 / (1 + exp(-y)) # sigmoid function
    y = y.ravel()
    return y

In [4]:
#Calculate an accuracy metric
def accuracy(predicted_y, true_y):
    true_y = np.array(true_y).ravel()
    counter = 0
    for i in range(len(true_y)):
        p_y = predicted_y[i]
        t_y = true_y[i]
        if (p_y>.5 and t_y == 1) or (p_y < .5 and t_y == 0):
            counter+=1
    counter = (counter/ len(true_y)) * 100
    return counter

In [5]:
training_data_x = pickle.load(open("TS-Train.pkl", "rb")).dropna(axis=1)
training_data_x = preprocessing.normalize(training_data_x,norm='max', axis=0)
training_data_x = training_data_x.transpose()
print(training_data_x.shape)
training_data_y = labels("../archive/exoTrain.csv")

#set hyperparameters & variables
learning_rate = 0.03
epochs = 500
display_step = 5
n_samples = training_data_x.shape[1]
col_num = training_data_x.shape[0]

X = tf.placeholder(tf.float32, [col_num, n_samples])
Y = tf.placeholder(tf.float32, [1, n_samples]) #resulting dimenstion of W*X matmul

(780, 5087)


In [6]:
# We want the weight vector to correspond one to one with every column
W = tf.Variable(tf.zeros([1,col_num], dtype=np.float32), name="weight")
b = tf.Variable(tf.zeros([1, ], dtype=np.float32), name="bias")

#matrix multiplication requires outer dimension of W to be equal to be equal to the inner dimension of X: 
# (1,col_num) & (col_num, num_samples) - this is why we transpose X
pred = tf.matmul(W, X) + b # y′(x,A,b)=Ax+b linear matrix equation

error = tf.reduce_sum((pred-Y)**2) / (n_samples * 2) #MSE

In [7]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

session = tf.Session()
session.run(tf.global_variables_initializer())

loss_arr = []
acc_arr = []

for t in range(epochs):
    
    _, current_loss, current_W, current_b = session.run([optimizer, error, W, b], feed_dict={
        X: training_data_x,
        Y: training_data_y
    })
    
    loss_arr.append(current_loss)
    acc_arr.append(accuracy(logistic_layer(np.dot(current_W,training_data_x) + current_b), training_data_y))
    
print("Optimization Finished!")

training_error = session.run(error, feed_dict={X: training_data_x, Y: training_data_y})
print("Training error=", training_error, "Weights=", session.run(W), "Bias=", session.run(b), '\n')

2021-12-15 08:13:31.148557: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-15 08:13:31.156959: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-15 08:13:31.157643: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-15 08:13:31.159336: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Optimization Finished!
Training error= 0.0034484838 Weights= [[-3.73252900e-03 -4.33570949e-06  0.00000000e+00  6.28784532e-04
   2.46082520e-04  1.96478417e-04 -1.59915740e-04  1.20353047e-03
  -1.96631555e-03  1.05226409e-05  2.46082520e-04  3.21609172e-04
   1.84754259e-04  2.00113154e-05  1.99263057e-04 -1.26304268e-03
   6.66505366e-04  2.14116008e-04 -1.59915740e-04 -1.99045098e-04
   1.31031836e-03  1.45867316e-03 -1.23173674e-03  9.02846456e-04
   9.03031207e-04 -3.61261726e-03 -3.61385010e-03  7.73209287e-03
   4.63829702e-03  3.10483010e-04  2.48880300e-04  1.22826721e-04
   2.37835804e-04 -3.14673554e-04 -3.29532661e-03  1.92452615e-04
   2.55540072e-04  1.12550879e-04 -1.86221005e-04 -3.88050481e-04
  -5.66278410e-04 -2.31588632e-03 -1.76650829e-05  0.00000000e+00
   7.24365644e-04  3.21609172e-04  3.21609172e-04  3.21609172e-04
   3.21609172e-04  3.21609172e-04  3.21609172e-04  3.21609172e-04
   3.21609172e-04  3.21609172e-04  3.21609172e-04  3.21609172e-04
   3.21609172e-

In [8]:
test_x = pickle.load(open("TS-Test.pkl", "rb")).dropna(axis=1)
test_x = preprocessing.normalize(test_x, norm='max', axis=0)
test_x = test_x.transpose()

test_y = labels("../archive/exoTest.csv")

predicted_y = np.dot(session.run(W), test_x) + session.run(b)
predicted_y = logistic_layer(predicted_y)

print("Accuracy percentage: ", accuracy(predicted_y, test_y), "%")

Accuracy percentage:  40.0 %


In [9]:
with open('tf-models/multi-lr.npy', 'wb') as f:
    np.save(f, session.run(W))
    np.save(f, session.run(b))
    np.save(f, loss_arr)
    np.save(f, acc_arr)
